# Вебинар 5. Ранжирование и Гибридные рекоммендательные системы

# 1. Ранжирование

### 1.1 BPR - Bayesian Personalized Ranking loss

$$loss = \sigma(p^{+} - p^{-})$$

1. Случайная пара user - купленный им item_i = позитивная пара
2. Для того же юзера сэмплируем item_j (не купил или вес в user-item матрице ниже, у item_i из 1.) - негативная пара
3. Прогнозы модели $p^{+}$, $p^{-}$ --> $p^{+} - p^{-}$
4. loss = $\sigma(p^{+} - p^{-})$ - это и есть наша ошибка
5. Обновляем эмбеддинги

### 1.2 WARP = Weighted Approximate-Rank Pairwise loss
Неплохое [объяснение](https://medium.com/@gabrieltseng/intro-to-warp-loss-automatic-differentiation-and-pytorch-b6aa5083187a) на игрушечном примере

$$loss = ln(\frac{K-1}{N})*(p^- - p^+)$$

1. Сэмплируем случайную позитивную пару юзер-item, прогноз модели для этой пары $p^+$
2. Сюмплируем далее до тех пор, пока не найдем негативную пару, где прогнозная вероятность $p^-$ > $p^+$
3. K - общее количество рекомендаций
4. N - кол-во сэмплирований (зависит от качества модели!)

**Note**  
loss зависит только от 2-ух сэмплированных пар, эмбеддинги только для этих пар будут обновлены

---

# 2. LightFM

Формула эмбеддинга в ALS (матричной факторизации)

$$user_{emb} = \theta_1 * i_1 + ... + \theta_n * i_n$$  
  
$$p = user_{emb} * item_{emb}^T + b_u + b_i$$

Формула в LightFM

$$user_{emb} = \theta_1 * i_1 + ... + \theta_n * i_n + \alpha_1 * f_1 + ... + \alpha_k * f_k$$  
  
$$p = user_{emb} * item_{emb}^T + b_u + b_i$$

### Зачем нужны гибридные системы?

*Холодный старт* - ситуация, в которой с товаром/юзером в матрице мало взаимодействий

Холодный старт для товаров - обычно введение нового товара (YouTube, Lamoda, hh.ru)  
Холодный старт для юзера - к нам пришел новый юзер  

Холодного юзера/товар можно описать фичами --> мы больше о нем знаем --> он уже не такой холодный

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

C:\ProgramData\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
from lightfm.evaluation import precision_at_k, recall_at_k

from metrics import precision_at_k as custom_precision, recall_at_k
from utils import prefilter_items

In [3]:
data = pd.read_csv('./data/retail_train.csv')

item_features = pd.read_csv('./data/product.csv')
user_features = pd.read_csv('./data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [5]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [6]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [7]:
user_features['age_desc'].unique()

array(['65+', '45-54', '25-34', '35-44', '19-24', '55-64'], dtype=object)

In [8]:
user_features['marital_status_code'].unique()

array(['A', 'U', 'B'], dtype=object)

In [9]:
user_features['household_size_desc'].unique()

array(['2', '3', '4', '1', '5+'], dtype=object)

## 1. Filter items

In [10]:
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [11]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

C:\Users\Nata\Desktop\webinar_5\webinar_5\utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 86865 to 5001


# 2. Prepare data set

## 2.1 Prepare csr train matrix

In [12]:
user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,117847,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15926775,15926844,15926886,15972074,15972298,15972565,15972790,16100266,16729299,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2.2 Prepare CSR test matrix

In [13]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit
test_user_item_matrix.head(2)

item_id,32392,34873,42852,43094,44522,45507,49812,53516,58612,62804,...,17284297,17284423,17291184,17291665,17320698,17320911,17321092,17328261,17329749,17330255
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## 3. Prepare user and item features

In [15]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
user_feat.shape

(2497, 7)

In [17]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
117847,450.0,NUTRITION,National,REFRIGERATED,SOY/RICE MILK,64 OZ
818981,194.0,GROCERY,National,COLD CEREAL,ALL FAMILY CEREAL,10.4 OZ


In [18]:
item_feat.shape

(5001, 6)

## Encoding features

In [19]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [20]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Init model

In [21]:
model = LightFM(no_components=40,
                loss='warp', # "logistic","bpr"
                learning_rate=0.01, 
                item_alpha=0.4,
                user_alpha=0.1, 
                random_state=42,
                k=5,
                n=15,
                max_sampled=100)

## Train

In [22]:
model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=20, 
          num_threads=20,
          verbose=True) 

Epoch: 100%|██████████| 20/20 [03:05<00:00,  9.29s/it]


# Getting embeddings

## вектора по пользователям

In [23]:
user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())

In [24]:
user_emb[0].shape # biases

(2497,)

In [25]:
user_emb[1].shape # users vectors

(2497, 40)

## вектора по товарам

In [26]:
item_emb = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values).tocsr())

In [27]:
item_emb[0].shape # biases

(5001,)

In [28]:
item_emb[1].shape # items vectors

(5001, 40)

# Evaluation -> Train precision

In [29]:
# мы можем использовать встроенные метрики lightFM
train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

print(f"Train precision {train_precision}")

Train precision 0.2941129505634308


# Grid Search

In [30]:
model = LightFM(no_components=10,
                loss='bpr',
                learning_rate=0.01, 
                item_alpha=0.4,
                user_alpha=0.1, 
                random_state=42,
                k=5,
                n=15,
                max_sampled=100)
model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=20, 
          num_threads=20,
          verbose=True)
# мы можем использовать встроенные метрики lightFM
train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()
    
print(f"Train precision {train_precision}")

Epoch: 100%|██████████| 20/20 [00:50<00:00,  2.55s/it]


Train precision 0.4385262429714203


In [31]:
for loss_ in ('bpr','warp'):
    model = LightFM(no_components=10,
                loss=loss_, # "logistic","bpr",'warp'
                random_state=42,
                k=5,
                n=15,
                max_sampled=100)
    model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=20, 
          num_threads=20,
          verbose=True)
    # мы можем использовать встроенные метрики lightFM
    train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()
    
    print(f"Train precision {train_precision}, Parameter - {loss_}")

Epoch: 100%|██████████| 20/20 [00:51<00:00,  2.59s/it]


Train precision 0.2020024061203003, Parameter - bpr


Epoch: 100%|██████████| 20/20 [00:55<00:00,  2.76s/it]


Train precision 0.3162195086479187, Parameter - warp


In [32]:
for l_r in np.arange(0.01, 0.1, 0.01):
    model = LightFM(no_components=10,
                loss='warp',
                learning_rate=l_r, 
                random_state=42,
                k=5,
                n=15,
                max_sampled=100)
    model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=20, 
          num_threads=20,
          verbose=True)
    # мы можем использовать встроенные метрики lightFM
    train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()
    
    print(f"Train precision {train_precision}, Parameter - {l_r}")

Epoch: 100%|██████████| 20/20 [00:53<00:00,  2.70s/it]


Train precision 0.28634360432624817, Parameter - 0.01


Epoch: 100%|██████████| 20/20 [00:54<00:00,  2.71s/it]


Train precision 0.29106929898262024, Parameter - 0.02


Epoch: 100%|██████████| 20/20 [00:54<00:00,  2.71s/it]


...

Train precision 0.3247096836566925, Parameter - 0.06999999999999999


Epoch: 100%|██████████| 20/20 [00:54<00:00,  2.73s/it]


Train precision 0.3153384029865265, Parameter - 0.08


Epoch: 100%|██████████| 20/20 [00:54<00:00,  2.73s/it]


Train precision 0.3114137053489685, Parameter - 0.09


In [33]:
for i_alpha in np.arange(0.1, 1.1, 0.1):
    for u_alpha in np.arange(0.1, 1.1, 0.1):
        model = LightFM(no_components=10,
                    loss='warp',
                    learning_rate=0.07, 
                    item_alpha=i_alpha,
                    user_alpha=u_alpha, 
                    random_state=42,
                    k=5,
                    n=15,
                    max_sampled=100)
        model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
              sample_weight=coo_matrix(user_item_matrix),
              user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
              item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
              epochs=20, 
              num_threads=20,
              verbose=True)
        # мы можем использовать встроенные метрики lightFM
        train_precision = precision_at_k(model, sparse_user_item, 
                                     user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                     item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                     k=5).mean()

        print(f"Train precision {train_precision}, Parameter item_alpha- {i_alpha}, user_alpha - {u_alpha}")

Epoch: 100%|██████████| 20/20 [00:53<00:00,  2.67s/it]


Train precision 0.2941129505634308, Parameter item_alpha- 0.1, user_alpha - 0.1


Epoch: 100%|██████████| 20/20 [00:54<00:00,  2.70s/it]


Train precision 0.2941129505634308, Parameter item_alpha- 0.1, user_alpha - 0.2


Epoch: 100%|██████████| 20/20 [00:54<00:00,  2.70s/it]


...

Train precision 0.14833801984786987, Parameter item_alpha- 1.0, user_alpha - 0.8


Epoch: 100%|██████████| 20/20 [00:33<00:00,  1.69s/it]


Train precision 0.2941129505634308, Parameter item_alpha- 1.0, user_alpha - 0.9


Epoch: 100%|██████████| 20/20 [00:33<00:00,  1.70s/it]


Train precision 0.2941129505634308, Parameter item_alpha- 1.0, user_alpha - 1.0


In [34]:
for l_r in np.arange(0.01, 0.1, 0.01):
    model = LightFM(no_components=10,
                loss='bpr',
                learning_rate=l_r,
                random_state=42,
                k=5,
                n=15,
                max_sampled=100)
    model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=20, 
          num_threads=20,
          verbose=True)
    # мы можем использовать встроенные метрики lightFM
    train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()
    
    print(f"Train precision {train_precision}, Parameter - {l_r}")

Epoch: 100%|██████████| 20/20 [00:51<00:00,  2.60s/it]


Train precision 0.20040050148963928, Parameter - 0.01


Epoch: 100%|██████████| 20/20 [00:51<00:00,  2.59s/it]


Train precision 0.20408493280410767, Parameter - 0.02


Epoch: 100%|██████████| 20/20 [00:51<00:00,  2.59s/it]


...

Train precision 0.1956748217344284, Parameter - 0.06999999999999999


Epoch: 100%|██████████| 20/20 [00:51<00:00,  2.58s/it]


Train precision 0.18309973180294037, Parameter - 0.08


Epoch: 100%|██████████| 20/20 [00:51<00:00,  2.58s/it]


Train precision 0.18590310215950012, Parameter - 0.09


In [35]:
for i_alpha in np.arange(0.1, 1.1, 0.1):
    for u_alpha in np.arange(0.1, 1.1, 0.1):
        model = LightFM(no_components=10,
                    loss='bpr',
                    learning_rate=0.03, 
                    item_alpha=i_alpha,
                    user_alpha=u_alpha, 
                    random_state=42,
                    k=5,
                    n=15,
                    max_sampled=100)
        model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
              sample_weight=coo_matrix(user_item_matrix),
              user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
              item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
              epochs=20, 
              num_threads=20,
              verbose=True)
        # мы можем использовать встроенные метрики lightFM
        train_precision = precision_at_k(model, sparse_user_item, 
                                     user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                     item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                     k=5).mean()

        print(f"Train precision {train_precision}, Parameter item_alpha- {i_alpha}, user_alpha - {u_alpha}")

Epoch: 100%|██████████| 20/20 [00:50<00:00,  2.55s/it]


Train precision 0.21097318828105927, Parameter item_alpha- 0.1, user_alpha - 0.1


Epoch: 100%|██████████| 20/20 [00:50<00:00,  2.55s/it]


Train precision 0.21113339066505432, Parameter item_alpha- 0.1, user_alpha - 0.2


Epoch: 100%|██████████| 20/20 [00:51<00:00,  2.55s/it]


...

Train precision 0.21305571496486664, Parameter item_alpha- 1.0, user_alpha - 0.8


Epoch: 100%|██████████| 20/20 [00:29<00:00,  1.45s/it]


Train precision 0.20288348197937012, Parameter item_alpha- 1.0, user_alpha - 0.9


Epoch: 100%|██████████| 20/20 [00:28<00:00,  1.43s/it]


Train precision 0.20264320075511932, Parameter item_alpha- 1.0, user_alpha - 1.0


Лучшее значение __Train precision__ - 0.4385262429714203, можнот получить при настройке гиперпараметров и для функции потерь __'bpr'__, и для __'warp'__.

In [36]:
model = LightFM(no_components=10,
                loss='warp',
                learning_rate=0.07, 
                item_alpha=0.2,
                user_alpha=0.3, 
                random_state=42,
                k=5,
                n=15,
                max_sampled=100)

model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=20, 
          num_threads=20,
          verbose=True)

Epoch: 100%|██████████| 20/20 [00:53<00:00,  2.70s/it]


In [37]:
# мы можем использовать встроенные метрики lightFM
train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()
    
print(f"Train precision {train_precision}")

Train precision 0.4385262429714203


# Predict

In [38]:
# подготавливаемм id для юзеров и товаров в порядке пар user-item
users_ids_row = data_train_filtered['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
items_ids_row = data_train_filtered['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)

In [39]:
users_ids_row[:10]

array([2371, 1363, 1363, 1363, 1363, 1171, 1171, 1171, 1171, 1171])

In [40]:
# модель возвращает меру/скор похожести между соответствующим пользователем и товаром
predictions = model.predict(user_ids=users_ids_row,
                            item_ids=items_ids_row,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=10)

In [41]:
# добавляем наш полученный скор в трейн датафрейм
data_train_filtered['score'] = predictions

In [42]:
# создаем предикт датафрейм в формате списка това
predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999].drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id']. \
            unique().reset_index()

In [43]:
# объединяем предикт и тест датасет для подсчета precision
df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')

In [44]:
df_result_for_metrics.head()

,user_id,actual,item_id
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1029743, 6034857, 857006, 952163, 1004906, 98..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 866211, 1127831, 965267, 899624, 874..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1070820, 1029743, 1126899, 866211, 1127831, 8..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1029743, 1126899, 1106523, 866211, 1127831, 8..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1106523, 1070820, 1029743, 6034857, 916122, 8..."


In [45]:
precision = df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean()
print(f"Precision: {precision}")

Precision: 0.14288998357963698


# Links

Neural networks for RS: http://d2l.ai/chapter_recommender-systems/mf.html

LigthFM -> https://arxiv.org/pdf/1507.08439.pdf

https://making.lyst.com/lightfm/docs/home.html


# Домашнее задание

#### 1) Прочитать статьи про BPR, WARP loss

#### 2) Сделать грид серч текущей модели, смотрите на метрику precision@5, считаем на тесте нашей функцией